In [1]:
tweets = read.csv("tweets.csv", stringsAsFactors=FALSE)

str(tweets)

'data.frame':	1181 obs. of  2 variables:
 $ Tweet: chr  "I have to say, Apple has by far the best customer care service I have ever received! @Apple @AppStore" "iOS 7 is so fricking smooth & beautiful!! #ThanxApple @Apple" "LOVE U @APPLE" "Thank you @apple, loving my new iPhone 5S!!!!!  #apple #iphone5S pic.twitter.com/XmHJCU4pcb" ...
 $ Avg  : num  2 2 1.8 1.8 1.8 1.8 1.8 1.6 1.6 1.6 ...


In [2]:
tweets$Negative = as.factor(tweets$Avg <= -1)

table(tweets$Negative)


FALSE  TRUE 
  999   182 

In [4]:
library(tm)
library(SnowballC)

Loading required package: NLP


In [5]:
corpus = VCorpus(VectorSource(tweets$Tweet)) 

In [6]:
corpus = tm_map(corpus, content_transformer(tolower))
corpus = tm_map(corpus, removePunctuation)
corpus = tm_map(corpus, removeWords, c("apple", stopwords("english")))
corpus = tm_map(corpus, stemDocument)

In [8]:
frequencies = DocumentTermMatrix(corpus)
findFreqTerms(frequencies, lowfreq=100)

[1] "iphon" "itun"  "new"

In [10]:
sparse = removeSparseTerms(frequencies, 0.995)
sparse

# Convert to a data frame

tweetsSparse = as.data.frame(as.matrix(sparse))

# Make all variable names R-friendly

colnames(tweetsSparse) = make.names(colnames(tweetsSparse))

# Add dependent variable

tweetsSparse$Negative = tweets$Negative

<<DocumentTermMatrix (documents: 1181, terms: 309)>>
Non-/sparse entries: 4669/360260
Sparsity           : 99%
Maximal term length: 20
Weighting          : term frequency (tf)

In [11]:
library(caTools)

set.seed(123)

split = sample.split(tweetsSparse$Negative, SplitRatio = 0.7)

trainSparse = subset(tweetsSparse, split==TRUE)
testSparse = subset(tweetsSparse, split==FALSE)

In [13]:
logit = glm(Negative ~ ., data=trainSparse, family=binomial)
prediction = predict(logit, newdata = testSparse, type = "response")

Warning message:
“glm.fit: algorithm did not converge”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = ifelse(type == :
“prediction from a rank-deficient fit may be misleading”

In [15]:
table(prediction > 0.5, testSparse$Negative)

       
        FALSE TRUE
  FALSE   252   24
  TRUE     48   31

In [16]:
(252 + 31) / nrow(testSparse)

[1] 0.7971831